# Boston Housing Classification SVM Evaluation

In [ ]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.functions import expr
from pyspark.sql.session import SparkSession
from pyspark.sql.types import BooleanType
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = translate_to_file_string("../data/Boston_Housing_Data.csv")

Spark session creation 

In [ ]:
spark = (SparkSession
       .builder
       .appName("BostonHoudingSVNEval")
       .getOrCreate())

DataFrame creation using an ifered Schema 

In [ ]:
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile) \
       .withColumn("CATBOOL", expr("CAT").cast(BooleanType()))
print(df.printSchema())

In [ ]:
splits = df.randomSplit([0.9, 0.1 ], 12345)
training = splits[0]
test = splits[1]

Prepare training and test data.

Data preprocessing

In [ ]:
featureCols = df.columns.copy()
featureCols.remove("MEDV")
featureCols.remove("CAT")
featureCols.remove("CATBOOL") 
print(featureCols)

assembler =  VectorAssembler(outputCol="features", inputCols=featureCols)

Build the evaluator

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="CAT",rawPredictionCol="rawPrediction", metricName="areaUnderROC")
#evaluator = MulticlassClassificationEvaluator(labelCol="CAT", predictionCol="prediction", metricName='weightedPrecision')

Support Vector Machine Classifier

In [ ]:
lsvc = LinearSVC(labelCol="CAT",aggregationDepth=2, featuresCol="features" ) 

Build the pipeline

In [ ]:
pipeline = Pipeline(stages= [assembler, lsvc] )

Build the paramGrid

In [ ]:
# TODO Add your settings there
paramGrid = ParamGridBuilder().addGrid(lsvc.maxIter, [100])\
                                 .addGrid(lsvc.regParam, [0.1]) \
                                 .build()

Build the CrossValidator 

In [ ]:
cvSVM = CrossValidator(estimator=pipeline, evaluator=evaluator, \
                          estimatorParamMaps=paramGrid, numFolds=5, parallelism=2)

Train the model 

In [ ]:
cvSVMModel = cvSVM.fit(training)

Test the model

In [ ]:
predictions = cvSVMModel.transform(test)
predictions.show()

# Evaluate the Model
## Area under ROC

In [ ]:
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))

## Confusion Matrix 

In [ ]:
predictionAndLabels = predictions.select("prediction", "CAT").rdd.map(lambda p: [p[0], float(p[1])]) # Map to RDD prediction|label
labels = predictionAndLabels.map(lambda x: x[1]).distinct().collect() # List of all labels 
metrics =  MulticlassMetrics(predictionAndLabels)

In [ ]:
# TODO print and evaluate on MulticlassMetrics metrics
# Confusion Matrix

## Statistics per label

In [ ]:
for label in  labels:
   print("Class %f precision = %f\n" % (label , metrics.precision(label))) 
  # TODO add additional statistics for the label (recall, ...)

## Weighted stats

In [ ]:
#TODO print weighted Stats 

In [ ]:
## Summary stats

In [ ]:
print(f"Recall = {metrics.recall(1.0)}")
print(f"Precision = {metrics.precision(1.0)}")
print(f"Accuracy = {metrics.accuracy}") 
print(f"F1 = {metrics.fMeasure(1.0)}")

In [ ]:
spark.stop()